Reference for valid city names in Houston Area:
https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Greater_Houston

Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected


In [1]:
from collections import defaultdict
import pprint
import xml.etree.cElementTree as ET

In [19]:
SAMPLE_FILE = "C:\Chaitanya\Jupyter\Data_Wrangling\Project_DataWrangling\houston_texas_sample.osm"
OSMFILE = "C:\Chaitanya\Jupyter\Data_Wrangling\Project_DataWrangling\houston_texas.osm"

In [3]:
def tag_city(file):
    tag = defaultdict(int)
    for event, elem in ET.iterparse(file):
        if (elem.tag == 'way'):
            for e in elem.iter("tag"):
                    if (e.attrib['k'] == 'addr:city'):
                        tag[e.attrib['v']] +=1
    print tag            
    return tag
tag_city(SAMPLE_FILE)

defaultdict(<type 'int'>, {'Needville': 2, 'Houston, Texas': 5, 'Pearland': 39, 'Magnolia': 11, 'Dickinson': 7, 'Sugarland': 2, 'Kingwood': 55, 'Texas City': 7, 'The Woodlands': 28, 'Richmond': 24, 'Plantersville': 1, 'Pasadena': 27, 'Baytown': 11, 'Bay City': 1, 'Cypress, TX': 2, 'Santa Fe, TX': 1, 'Seabrook': 30, 'La Porte': 1, 'Klein': 43, 'Pasadena, TX': 1, 'Atascocita': 1, 'Nassau Bay': 1, 'Cypress': 39, 'Kemah': 1, 'Angleton, TX': 1, 'Hockley': 2, 'Shenandoah': 2, 'Angleton,TX': 1, 'Rosharon': 4, 'West University Place': 6, 'Conroe': 3, 'Missouri City': 24, 'Clear Lake Shores': 1, 'Missouri City, TX': 2, 'League City': 25, 'Tomball': 245, 'Bay City, TX': 1, 'West Columbia, TX': 1, 'Stafford': 13, 'Tomball, Tx': 58, 'Lake Jackson': 14, 'Freeport': 1, 'Spring, TX': 2, 'Humble, TX': 1, 'Katy': 430, 'Fort Bend': 2, 'Meadows Place': 1, 'Humble': 55, 'Clute': 3, 'Galveston Island': 1, 'Hedwig Village': 1, 'katy': 1, 'Alvin': 4, 'Bellaire': 13, '77386': 1, 'Deer Park': 2, 'Webster': 4, 

defaultdict(int,
            {'77386': 1,
             'Alvin': 4,
             'Alvin, TX': 1,
             'Angleton': 1,
             'Angleton, TX': 1,
             'Angleton,TX': 1,
             'Atascocita': 1,
             'Bay City': 1,
             'Bay City, TX': 1,
             'Baytown': 11,
             'Bellaire': 13,
             'Brazoria': 1,
             'Channelview': 1,
             'Clear Lake Shores': 1,
             'Clute': 3,
             'Conroe': 3,
             'Crosby': 1,
             'Cypress': 39,
             'Cypress, TX': 2,
             'Deer Park': 2,
             'Dickinson': 7,
             'Fort Bend': 2,
             'Freeport': 1,
             'Fresno': 3,
             'Friendswood': 8,
             'Friendswood, TX': 1,
             'Galveston': 684,
             'Galveston Island': 1,
             'Hedwig Village': 1,
             'Hockley': 2,
             'Houston': 2428,
             'Houston, TX': 18,
             'Houston, Texas': 5,
   

In [24]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

#street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Alvin", "Angleton", "Atascocita", "Bay City", "Baytown", "Bellaire", "Brazoria", "Channelview", "Clear Lake Shores", 
            "Clute", "Conroe", "Crosby", "Cypress", "Deer Park", "Dickinson", "Freeport", "Fresno'", "Friendswood", 
            "Galveston", "Hedwig Village", "Hockley", "Houston", "Humble", "Katy", "Kemah", "Kingwood", "Klein", "La Porte", "LaMarque",
            "Lake Jackson", "League City", "Liberty", "Magnolia", "Meadows Place", "Missouri City", "Nassau Bay", "Needville", 
            "Pasadena", "Pearland", "Porter", "Richmond", "Rosenberg", "Santa Fe", "Seabrook", "Shenandoah", "Spring", "Stafford", 
            "Sugar Land", "Texas City", "The Woodlands", "Tomball", "Webster", "West Columbia", "West University Place", "Winnie"]


mapping = {
    "Laks Jackson": "Lake Jackson", "Houson": "Houston", "Galveston Island": "Galveston", "The Woodlands": "Woodlands",\
    "Sugarland": "Sugar Land", "West University": "West University Place", "Dickenson": "Dickinson"
}

def audit_city(invalid_city_names, city_name):
    if city_name not in expected:
        invalid_city_names[city_name] +=1
    return invalid_city_names


def is_city_name(elem):
    return (elem.attrib['k'] == "addr:city")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    invalid_city_names = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_city_name(tag):
                    audit_city(invalid_city_names, tag.attrib['v'])
    osm_file.close()
    return invalid_city_names


def correct_case(name):
    # This function is used to correct the case of city names
    if (name.isupper()):
        bettername = name.title()
        # convert UPPER case names to CAMEL case
    elif (name.islower()):
        bettername = name.title()
        # convert LOWER case names to CAMEL case
    else:
        bettername = name
    
    return bettername

def truncate_extension(name):
    # This function will return the name, truncating the part of the name after ',' or '#'
    # The objective of this function is to remove extensions of city/street names (like 'Tx' or 'Texas') and 
    # any house numbers or suite numbers following city/street names
    if (',' in name):
        bettername = name[:name.index(',')]
        # return the part of the name before ','
    elif ('#' in name):
        bettername = name[:name.index('#')]
        # return the part of the name before '#'
    else:
        bettername = name
    return bettername

def update_name(name, mapping):
    # This function is used to correct a few spellings in city names and to maintain consistency in city names
    if (name in mapping.keys()):
        bettername = mapping[name]
    else:
        bettername = name
    return bettername
    

if __name__ == '__main__':
    invalid_city = audit(OSMFILE)
    pprint.pprint(dict(invalid_city))
    for city, times in invalid_city.iteritems():
        better_name1 = correct_case(city)
        better_name2 = truncate_extension(better_name1)
        better_name3 = update_name(better_name2, mapping)
        print city, "=>", better_name3

{'77386': 1,
 'Alvin, TX': 1,
 'Angleton, TX': 1,
 'Angleton,TX': 1,
 'Bay City, TX': 2,
 'Beasley': 1,
 'Bellaire, TX': 1,
 'Crystal Beach': 2,
 'Cypress, TX': 4,
 'DEER PARK': 2,
 'Dickenson': 1,
 'Dickinson, Tx': 1,
 'El Lago': 1,
 'Fort Bend': 2,
 'Fresno': 3,
 'Friendswood, TX': 2,
 'Fulshear': 1,
 'Galveston Island': 1,
 'HOUSTON': 5,
 'Hempstead': 1,
 'Houson': 1,
 'Houston, TX': 46,
 'Houston, Texas': 7,
 'Huffman': 1,
 'Humble, TX': 2,
 'Jersey Village': 1,
 'KATY': 1,
 'Katy, TX': 6,
 'Kendleton': 3,
 'Kingwood, TX': 1,
 'LAKE JACKSON': 3,
 'La Marque': 1,
 'Lake Jackson, TX': 1,
 'Laks Jackson': 2,
 'League City, TX': 3,
 'Little York': 1,
 'MAGNOLIA': 2,
 'Missouri City, TX': 4,
 'Mont Belvieu': 1,
 'Navasota': 1,
 'New Caney': 4,
 'Pasadena, TX': 2,
 'Pearland, TX': 2,
 'Plantersville': 1,
 'Rosharon': 6,
 'San Leon': 1,
 'Santa Fe, TX': 1,
 'Sealy': 1,
 'South Houston': 1,
 'Spring, TX': 8,
 'Sugar Land, TX': 5,
 'Sugarland': 3,
 'TEXAS CITY': 2,
 'The Woodlands, TX': 2,
